In [1]:
%%javascript
$('#appmode-leave').hide();                          // Hides the edit app button.
$('#appmode-busy').hide();                           // Hides the kernel busy indicator.
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false; // disable scrolling
}

<IPython.core.display.Javascript object>

In [2]:
# Google Forms / Sheets Code
import urllib.request
from bs4 import BeautifulSoup
import requests, warnings

def get_questions(in_url):
    res = urllib.request.urlopen(in_url)
    soup = BeautifulSoup(res.read(), 'html.parser')
    get_names = lambda f: [v for k,v in f.attrs.items() if 'label' in k]
    get_name = lambda f: get_names(f)[0] if len(get_names(f))>0 else 'unknown'
    all_questions = soup.form.findChildren(attrs={'name': lambda x: x and x.startswith('entry.')})
    return {get_name(q): q['name'] for q in all_questions}
def submit_response(form_url, cur_questions, verbose=False, **answers):
    submit_url = form_url.replace('/viewform', '/formResponse')
    form_data = {'draftResponse':[],
                'pageHistory':0}
    for v in cur_questions.values():
        form_data[v] = ''
    for k, v in answers.items():
        if k in cur_questions:
            form_data[cur_questions[k]] = v
        else:
            warnings.warn('Unknown Question: {}'.format(k), RuntimeWarning)
    if verbose:
        print(form_data)
    user_agent = {'Referer':form_url,
                  'User-Agent': "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.52 Safari/537.36"}
    return requests.post(submit_url, data=form_data, headers=user_agent)

In [3]:
import os
import ipywidgets as ipw
from glob import glob
import json
import pandas as pd
import numpy as np
from PIL import Image
from itertools import cycle
from io import BytesIO
from time import time
from collections import namedtuple
from IPython.display import display
USERNAME=os.environ.get('APPMODE_USER', 'Not logged in')
HOSTNAME=os.environ.get('HOSTNAME', 'anon')
BUTTONS_PER_ROW = 3
BUTTON_WIDTH = "150px"
UNKNOWN_OPTION = 'Unknown'

/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
with open('task.json', 'r') as f:
    annotation_task = json.load(f)
    annot_df = pd.DataFrame(annotation_task['dataset']['dataframe'])
print('Loaded annotation task', annot_df.shape[0], 'images')

Loaded annotation task 110 images


In [5]:
image_path_dict = {c_path: os.path.join(annotation_task['dataset']['base_image_directory'], 
                                    c_path)
                                    for c_path in annot_df[annotation_task['dataset']['image_path']]}
category_ids = sorted(annot_df[annotation_task['dataset']['output_labels']].unique().tolist())
if UNKNOWN_OPTION is not None:
    category_ids += [UNKNOWN_OPTION]   

In [6]:
mc_question = namedtuple('MultipleChoiceAnswer', ['answer', 'question'])
TaskResult = namedtuple('TaskResult', ['annotation_mode', 'task', 'item_id', 'label'])


class SimpleImageViewer:
    def __init__(self):
        self.cur_image_view = ipw.Image(layout=ipw.Layout(width="512px"),  
                  disabled=True)
        
    def load_image_path(self, path):
        c_img = Image.open(path)
        bio_obj = BytesIO()
        c_img.save(bio_obj, format='png')
        bio_obj.seek(0)
        self.cur_image_view.value=bio_obj.read()
        
    def get_widget(self):
        return self.cur_image_view


class MultipleChoiceQuestion:
    def __init__(self, question, labels):
        self.question = question
        self.labels = labels
        self.question_box = ipw.HTML(value='<h2>{}</h2>'.format(question))
        self._make_buttons(labels)
        self.submit_func = None
        
    def on_submit(self, submit_func):
        self.submit_func = submit_func
        
    def mk_btn(self, description):
        btn = ipw.Button(description=description, 
                         layout=ipw.Layout(width=BUTTON_WIDTH))
        def on_click(btn):
            if btn is not None and self.submit_func is not None:
                self.submit_func(mc_question(btn.description, self.question))
        btn.on_click(on_click)
        return btn
    
    def get_widget(self):
        return ipw.VBox([self.question_box]+self.button_rows)
    
    def _make_buttons(self, button_ids):
        self.button_rows = []
        c_row = []
        for i, but_name in enumerate(button_ids, 1):
            c_row+=[self.mk_btn(but_name)]
            if (i % BUTTONS_PER_ROW)==0:
                self.button_rows+=[ipw.HBox(c_row)]
                c_row=[]
        self.button_rows+=[ipw.HBox(c_row)]

class MultiClassTask:
    def __init__(self, labels, images, seed=None):
        self.labels = labels
        self.images = images
        self.image_keys = sorted(list(images.keys()))
        self.submit_event = None
        self.task_widget = SimpleImageViewer()
        self.answer_widget = MultipleChoiceQuestion('Select the most appropriate label for the given image', labels)
        self.answer_widget.on_submit(lambda x: self._local_submit(x))
        self.current_image_id = None
        self.set_seed(seed)
        self._local_submit(mc_question(question='', answer=''))
    
    def get_widget(self):
        return ipw.HBox([self.task_widget.get_widget(), self.answer_widget.get_widget()])
                        
    def set_seed(self, seed):
        if seed is not None:
            np.random.seed(seed)
            
    def _ipython_display_(self):
        display(self.get_widget())
    
    def on_submit(self, on_submit):
        self.submit_event = on_submit
    
    def _local_submit(self, mc_answer):
        c_task = TaskResult(annotation_mode='MultiClass', 
                            task=','.join(self.labels),
                            item_id=self.current_image_id,
                            label=mc_answer.answer
                  )
        
        # get next question
        image_key = np.random.choice(self.image_keys)
        # update image
        self.task_widget.load_image_path(self.images[image_key])
        
        self.current_image_id = image_key
        # submit results to backend
        
        if self.submit_event is not None:
            self.submit_event(c_task)

In [7]:
FORM_URL = annotation_task['google_forms']['form_url']
anno_questions = get_questions(FORM_URL)
def submit_label(task_result):
    """
    should be linked to some kind of database
    """
    print('Submitting', task_result.label, 'for', task_result.item_id, 'from', USERNAME, 'part of', HOSTNAME, '@', time())
    submit_response(FORM_URL, anno_questions, 
                    annotator=USERNAME, 
                    session=HOSTNAME, 
                    time = time(),
                    item_id = task_result.item_id,
                    label = task_result.label,
                    task = task_result.task,
                    annotation_mode=task_result.annotation_mode
                   )

In [8]:
title_box = ipw.HTML(value=f'<h1> Welcome {USERNAME}</h1>')
task_widget = MultiClassTask(category_ids, image_path_dict)
task_widget.on_submit(submit_label)
ipw.VBox([title_box, task_widget.get_widget()])